In [2]:
import time
import requests
import pandas as pd

In [19]:
okx_protocol_urls = {"lido": [10002],
                         "renzo": [22750],
                         "eigenlayer": [22004, 21999, 22003, 22000, 22001],
                         "etherfi": [22850],
                         "kelp": [22700],
                         "swell": [21915],
                         "zircuit": [30813, 30802, 30801, 30812],
                         }
defilama_karak_pools = ["8903d930-5d19-4d21-b0de-f051cba55a03", "c97780f7-fc3a-475c-9492-ce1e743016fb"]


def get_mitosis_apy():
    pass


def get_elixir_apy():
    apy_url = "https://metrics-api.trusted-mainnet.elixir.xyz/metrics/tvl-apy"
    response = requests.get(url=apy_url)
    if response.status_code != 200:
        return False
    exchanges = response.json().get("data").get("tvl_per_exchange")

    total_apy = (exchanges.get("RABBITX").get("token_pairs").get("ETH_PERP").get("pool_apy")
                    + exchanges.get("APEX").get("token_pairs").get("ETH_USDC").get("pool_apy")) / 2
    return {"elixir": round(total_apy, 2)}


def get_karak_apy(karak_pools: list):
    total_apy = 0
    for pool in karak_pools:
        req = requests.get(f"https://yields.llama.fi/chart/{pool}").json()['data'][0]
        total_apy += float(req['apy'])
    return {"karak": round(total_apy / len(karak_pools), 2)}


def get_protocols_apr(protocols: dict):
    result = {}
    for protocol, tokens in protocols.items():
        apy_over_protocol = 0
        for token in tokens:
            base_url = f"https://www.okx.com/priapi/v1/invest/web/products/{token}/rate/chart?type=1&investmentType=0&t={time.time}"
            response = requests.post(url=base_url)
            if response.status_code == 200:
                apy_over_protocol += float(response.json().get("data")[-1]['rate']) * 100 if response.json().get(
                    "data") else 0.01
        result.update({protocol: round(apy_over_protocol / len(tokens), 2)})
    result.update(get_karak_apy(defilama_karak_pools))
    result.update(get_elixir_apy())
    return result

print(get_protocols_apr(protocols=okx_protocol_urls))

{'lido': 3.24, 'renzo': 2.46, 'eigenlayer': 3.28, 'etherfi': 2.24, 'kelp': 1.93, 'swell': 3.25, 'zircuit': 0.01, 'karak': 23.24, 'elixir': 11.46}


In [18]:
whalesmarket_offers = {"renzo": "Renzo",
                       "eigenlayer": "EigenLayer",
                       "etherfi": "ether.fi",
                       "kelp": "KelpDAO",
                       "swell": "Swell",
                       "elixir": "Elixir",
                       "karak": "Karak"}

def get_eth_price():
    url = "https://api.coingecko.com/api/v3/simple/price?ids=ethereum&vs_currencies=usd"
    response = requests.get(url)
    if response.status_code != 200:
        return 3808.53
    data = response.json()
    return data['ethereum']['usd']

def get_points(whales_symbols: dict):
    points_amount = [1000, 2000, 5000, 10000, 50000, 100000]
    eth_price = get_eth_price()
    points_amount_price = {}
    for protocol, symbol in whales_symbols.items():
        points_amount_price.update({protocol: []})
        base_url = f"https://api-v2.whales.market/v2/offers?take=30&page=1&type=sell&full_match=&symbol={symbol}&status=open&min_price=&max_price=&sort_price=DESC&category_token=point_market&chains=1"
        response = requests.get(url=base_url)
        if response.status_code != 200 or not response.json().get("data").get("list"):
            continue
        df = pd.DataFrame(response.json().get("data").get("list"))
        df.drop(set(df.columns) - {'offer_price_usd', 'ex_token__price', 'total_amount'}, axis=1, inplace=True)
        df = df.sort_values("offer_price_usd")
        for p_am in points_amount:
            apy = 0
            price = 0
            usd_amount = 0
            total_points_amount = p_am
            for index, row in df.iterrows():
                price = row["offer_price_usd"]
                if p_am < row["total_amount"]:
                    usd_amount += p_am * price
                    price = usd_amount / total_points_amount
                    apy = round((price * 365 * 24 / eth_price) * 100, 2)
                    break
                else:
                    p_am -= row["total_amount"]
                    usd_amount += row["total_amount"] * price
                if index == df.index[-1]:
                    total_points_amount-= p_am
                    price = usd_amount / total_points_amount
                    apy = round((price * 365 * 24 / eth_price) * 100, 2)
                    break
            points_amount_price[protocol].append({"points_amount": total_points_amount,
                                                    "price": price,
                                                    "apy": apy})

    return points_amount_price

print(get_points(whalesmarket_offers))

{'renzo': [{'points_amount': 1000, 'price': 0.19, 'apy': 43.72}, {'points_amount': 2000, 'price': 0.19, 'apy': 43.72}, {'points_amount': 5000, 'price': 0.19, 'apy': 43.72}, {'points_amount': 10000, 'price': 0.19, 'apy': 43.72}, {'points_amount': 39500.0, 'price': 0.2685718429508987, 'apy': 61.8}, {'points_amount': 39500.0, 'price': 0.2685718429508987, 'apy': 61.8}], 'eigenlayer': [{'points_amount': 1000, 'price': 0.24013476715742835, 'apy': 55.26}, {'points_amount': 2000, 'price': 0.24013476715742835, 'apy': 55.26}, {'points_amount': 5000, 'price': 0.24013476715742835, 'apy': 55.26}, {'points_amount': 10000, 'price': 0.24013476715742835, 'apy': 55.26}, {'points_amount': 50000, 'price': 0.2410546318537859, 'apy': 55.47}, {'points_amount': 68398.0, 'price': 0.25047720693587533, 'apy': 57.64}], 'etherfi': [], 'kelp': [{'points_amount': 1000, 'price': 0.0003033017779826, 'apy': 0.07}, {'points_amount': 2000, 'price': 0.0003033017779826, 'apy': 0.07}, {'points_amount': 5000, 'price': 0.0003

In [ ]:
def get_erc20_tokens(api_key):
    url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/map"
    params = {
        'start': '1',
        'limit': '5000',
        'sort': 'cmc_rank',
        'cryptocurrency_type': 'tokens'
    }
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key
    }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    tokens = [token for token in data['data'] if 'ETH' in token['platform']]
    return tokens

api_key = 'your_api_key_here'
tokens = get_erc20_tokens(api_key)
for token in tokens:
    print(f"Name: {token['name']}, Symbol: {token['symbol']}, Address: {token['platform']['token_address']}")

# Replace 'your_api_key_here' with your actual API key.

In [3]:
import requests
from bs4 import BeautifulSoup


def get_erc20_tokens(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.post(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    tokens = []
    token_mapping = {}
    table_body = soup.find('tbody', {'class':'align-middle'})
    if table_body:
        rows = table_body.find_all('tr')
        for row in rows:
            columns = row.find_all('td')
            if len(columns) >= 2:  # Ensure there are enough columns
                token_name = columns[1].text.strip().replace('\n', ' ')  # Assuming the token name is in the first column
                contract_address = columns[1].contents[0].attrs.get('href').split('/')[-1] # Assuming the contract address is in the second column
                tokens.append({'token': token_name, 'contract_address': contract_address})
                token_mapping.update({token_name.split(' ')[-1].strip('()'): contract_address})
    return tokens, token_mapping

url = 'https://etherscan.io/tokens'  # Replace with the actual URL
tokens, token_mapping = get_erc20_tokens(url)
print(len(tokens))
print(tokens)
print(token_mapping)


50
[{'token': 'Tether USD (USDT)', 'contract_address': '0xdac17f958d2ee523a2206206994597c13d831ec7'}, {'token': 'BNB (BNB)', 'contract_address': '0xB8c77482e45F1F44dE1745F52C74426C631bDD52'}, {'token': 'stETH (stETH)', 'contract_address': '0xae7ab96520de3a18e5e111b5eaab095312d7fe84'}, {'token': 'USDC (USDC)', 'contract_address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}, {'token': 'XRP (IBC) (XRP)', 'contract_address': '0x628f76eab0c1298f7a24d337bbbf1ef8a1ea6a24'}, {'token': 'Wrapped TON Coin (TONCOIN)', 'contract_address': '0x582d872a1b094fc48f5de31d3b73f2d9be47def1'}, {'token': 'SHIBA INU (SHIB)', 'contract_address': '0x95aD61b0a150d79219dCF64E1E6Cc01f0B64C4cE'}, {'token': 'Wrapped Ether (WETH)', 'contract_address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'}, {'token': 'Wrapped BTC (WBTC)', 'contract_address': '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599'}, {'token': 'ChainLink Token (LINK)', 'contract_address': '0x514910771af9ca656af840dff83e8264ecf986ca'}, {'token': 'Uniswap 

In [2]:
from pydantic import BaseModel


class Erc20Token(BaseModel):
    name: str
    symbol: str
    address: str


In [4]:
import requests
import json

In [5]:
def get_erc_20_tokens() -> list[Erc20Token]:
    url = 'https://api.coinranking.com/v2/coins'
    response = requests.get(url)

    result = json.loads(response.text)
    coins = result['data']['coins']

    filter_erc = lambda coin: any(address.startswith('ethereum/') for address in coin['contractAddresses'])
    map_to_eth_address = lambda coin: next((addr.split('/')[1] for addr in coin['contractAddresses'] if addr.startswith('ethereum/')), None)
    map_to_token = lambda coin: Erc20Token(
        name=coin['name'],
        symbol=coin['symbol'],
        address=map_to_eth_address(coin)
    )
    return list(map(map_to_token, filter(filter_erc, coins)))

print(get_erc_20_tokens())
len(get_erc_20_tokens())


[Erc20Token(name='Tether USD', symbol='USDT', address='0xdac17f958d2ee523a2206206994597c13d831ec7'), Erc20Token(name='BNB', symbol='BNB', address='0xb8c77482e45f1f44de1745f52c74426c631bdd52'), Erc20Token(name='Lido Staked Ether', symbol='stETH', address='0xae7ab96520de3a18e5e111b5eaab095312d7fe84'), Erc20Token(name='USDC', symbol='USDC', address='0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'), Erc20Token(name='Toncoin', symbol='TON', address='0x582d872a1b094fc48f5de31d3b73f2d9be47def1'), Erc20Token(name='Shiba Inu', symbol='SHIB', address='0x95aD61b0a150d79219dCF64E1E6Cc01f0B64C4cE'), Erc20Token(name='Wrapped liquid staked Ether 2.0', symbol='wstETH', address='0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0'), Erc20Token(name='Wrapped Ether', symbol='WETH', address='0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'), Erc20Token(name='Wrapped BTC', symbol='WBTC', address='0x2260fac5e5542a773aa44fbcfedf7c193bc2c599'), Erc20Token(name='Chainlink', symbol='LINK', address='0x514910771af9ca656af840dff83e

27